# Calcul de la consommation globale

On commence par charger les données, on les affiche, on les trie et on fait une première synthèse des consommation par moteur et par vol.

L'idées est de construire une table résumant les données de chaque vol et la consommation correspondate de chaque moteur.

In [1]:
import os
from glob import glob
import pandas as pd
import statsmodels.formula.api as smf
import tabata as tbt
from tabata import Opset
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
%reload_ext autoreload
%autoreload 2

## 1. Récupération et affichage des données.

On affiche les données de manière à mieux comprendre comment définir les phases de vol.
Pour simplifier les accès j'utilise l'Opset de tabata.

In [2]:
datadir = "../data/"
datafiles = glob(os.path.join(datadir,'*.h5'))

In [3]:
datafiles

['../data/Aircraft_01.h5', '../data/Aircraft_02.h5', '../data/Aircraft_03.h5']

In [4]:
sorting = lambda name: int(name[8:])
ds = Opset(datafiles[0], sortkey=sorting)
ds.plotc()

### Recherche d'un moyen d'identifier les phases de vol

Un affichage interactif avec des variables en parallèle de l'altitude.

In [5]:
from ipywidgets import interact, widgets

def mydoubleplot(varname, record):
    df = ds[record]
    tbt.doubleplot(df[varname], df["N2_1 [% rpm]"], title="Vol {}".format(record))

interact(mydoubleplot, varname=ds.df.columns, record=widgets.IntSlider(0,0,len(ds)-1,1)) ; #range(len(ds))) ;

interactive(children=(Dropdown(description='varname', options=('ALT [ft]', 'EGT_1 [deg C]', 'EGT_2 [deg C]', '…

In [6]:
LB2KG = 0.453592  # kg/lb 
KDENS = 0.73      # kg/l
L = []
for file in datafiles:
    ac = 'AC'+file[-5:-3]
    ds = Opset(file,sortkey=sorting) # Ne pas oublier de trier les enregistrements.
    for i,df in enumerate(ds):
        if len(df.columns)>0 and "ALT [ft]" in df.columns:
            alt = df["ALT [ft]"]
            Alt_max = max(alt)
            if Alt_max > 20000:
                mc = df["M [Mach]"]
                # Il faudra peut-être séparer la montée et la descente de la croisière.
                df1 = df[mc>0.1]
                # Quand il faudra rajouter les phases au sol.
                #i0 = df1.index[0]
                #i1 = df1.index[-1]
                #taxi_o = df[:i0-1]
                #taxi_i = df[i1+1:]
                Leg = len(df1)/3600.0 # En h
                Q1 = df1["Q_1 [lb/h]"].sum()/3600.0 # en lb
                Q2 = df1["Q_2 [lb/h]"].sum()/3600.0
                M1 = Q1*LB2KG # En kg
                M2 = Q2*LB2KG
                Masse = M1+M2
                C1 = M1/KDENS # En l
                C2 = M2/KDENS
                Mach_max = df1["M [Mach]"].max()
                L += [[ac, 'Left', i, Leg, Alt_max, Mach_max, Masse, C1]]
                L += [[ac, 'Right', i, Leg, Alt_max, Mach_max, Masse, C2]]
dft = pd.DataFrame(L,columns = ['AC', 'ENG', 'Flight', 'Leg', 'Alt_max', 'Mach_max', 'Masse', 'Conso'])

In [7]:
dft

,AC,ENG,Flight,Leg,Alt_max,Mach_max,Masse,Conso
0,AC01,Left,0,1.619722,39439.264501,0.640898,3725.812542,2547.848842
1,AC01,Right,0,1.619722,39439.264501,0.640898,3725.812542,2556.003956
2,AC01,Left,1,1.698056,39457.811063,0.649114,3873.297269,2652.136497
3,AC01,Right,1,1.698056,39457.811063,0.649114,3873.297269,2653.750172
4,AC01,Left,2,1.548889,40825.620025,0.624464,3395.975242,2326.615381
...,...,...,...,...,...,...,...,...
5989,AC03,Right,999,1.132222,37621.701406,0.591598,2481.130405,1724.592070
5990,AC03,Left,1000,1.423889,36471.814551,0.632681,2990.838764,2019.218569
5991,AC03,Right,1000,1.423889,36471.814551,0.632681,2990.838764,2077.820834
5992,AC03,Left,1001,1.110278,37134.854149,0.599814,2425.202981,1639.174987


### Affichage de la consommation en fonction de la durée du vol.

On présente une relation entre durée du vol et consommation.
La variance résiduelle peut eut-être être encore réduite en travaillant sur la montée et la descente séparemment de la croisière.

In [8]:
import plotly.express as px

In [9]:
fig = px.scatter(dft, x="Leg", y="Conso", color="AC", symbol="ENG", 
           hover_data=dict(AC=True, ENG=True, Flight=True, Leg=False, Conso=False), 
           opacity=0.4,
           title="Consommation en fonction de la durée de vol",
           labels={"Conso" : "Consommation (litres)", "Leg" : "Durée du vol (heures)"})
fig.update_traces(marker=dict(size=12))

Cette figure présente le calcul de la consommation en fonction de la durée des vols. Les avions sont représentés par des couleurs différentes, le moteur gauche est un point rond et le droit un losange. Cet affichage est interactif, en zoomant sur une durée particulière on se rend compte d'une variance importante.

In [10]:
# On focalise sur une durée particulière.
fig = px.scatter(dft, x="Leg", y="Conso", color="AC", symbol="ENG", 
           hover_data=dict(AC=True, ENG=True, Flight=True, Leg=False, Conso=False), 
           opacity=0.4,
           title="Consommation en fonction de la durée de vol",
           labels={"Conso" : "Consommation (litres)", "Leg" : "Durée du vol (heures)"})
fig.update_traces(marker=dict(size=12))
fig.update_layout(xaxis_range=[1.96,2.1], yaxis_range=[2400, 3500])

Notons que sur cette image nous observons un écart de plus de 1000 litres de carburant pour une durée d'environ 2h de vol.

## 2. Analyse séparée de chaque phase de vol.

Une fonction écrite par Dinh permet de récupérer chacune de s phases de vols indépendamment.
Pour illustrer cette fonction nous allors créer un nouveau Opset qui contiendra les phases de vols stockées sous forme de colonnes booléennes.

J'ai créé un fichier `__init__.py` pour pouvoir facilement importer les éléments de codes sans faire d'import local.

In [45]:
from M2DProjet2324 import detect_phase
detect_phase

<function M2DProjet2324.utils.detect_phase(df, altitude_threshold=0.95)>

In [59]:
# Rappel : ds contient les vols du premier avion.
ds.rewind()
altitude_threshold = 0.95 # (0.95) Valeur par défaut fixée par Dinh.
storename = os.path.basename(ds.storename)
newstore = os.path.join(datadir,'out',storename[:-3]+'_phases.h5')
ds2 = Opset(newstore).clean()

# Je reprend le début du code de la fonction get_consumption...
for i, df in enumerate(ds):

        if len(df.columns)>0 and "ALT [ft]" in df.columns:
            
            alt = df["ALT [ft]"]
            Alt_max = max(alt)

            if Alt_max > 20000:
                
                ## selecting phase index
                phases = detect_phase(df, altitude_threshold) # phases = taxi1_idx, climb_idx, cruise_idx, descend_idx, taxi2_idx

                if phases == False:
                    print(i)
                    continue
                else:
                    taxi1_idx, climb_idx, cruise_idx, descend_idx, taxi2_idx = phases
                    df['TAXI1'] = (df.index>=taxi1_idx[0]) & (df.index<taxi1_idx[1])
                    df['CLIMB'] = (df.index>=climb_idx[0]) & (df.index<climb_idx[1])
                    df['CRUISE'] = (df.index>=cruise_idx[0]) & (df.index<cruise_idx[1])
                    df['DESCEND'] = (df.index>=descend_idx[0]) & (df.index<descend_idx[1])
                    df['TAXI2'] = (df.index>=taxi2_idx[0]) & (df.index<taxi2_idx[1])
                    ds2.put(df)

Record is incomplete
560


In [62]:
ds2.rewind().plotc("CRUISE")

**Pour les étapes suivantes**

1. Faire un test statistique validant ce modèle.
2. Les résidus sont-ils gaussiens ?
3. Estimer les résidus et la variance globale.
4. Cette variance dépend elle de la durée du vol.
5. Améliorer l'estimation en rajoutant des facteurs ?
6. Montrer quels facteurs sont pertinents.
7. A-t-on des dépendances temporelles résiduelles ?